In [1]:
from deap import base, creator, gp, tools, algorithms
import operator
import random
import numpy as np
import time
import uuid
import csv
import os

In [2]:
POP_SIZE = 100
N_EVALS = 100000
N_REPLICATES = 30
MAX_TREE_HEIGHT = 25
SF_PEN_LAM = 0.1

time_str = time.strftime('%Y%m%d_%H%M%S')
out_dir = f'GP_results_{time_str}'
os.makedirs(out_dir, exist_ok=True)
csv_path = os.path.join(out_dir, 'GP_results.csv')

In [3]:
def protectedDiv(a, b):
    try:
        if b == 0:
            return 1.0
    except Exception:
        pass
    try:
        return a / b
    except ZeroDivisionError:
        return 1.0

In [4]:
pset_sr = gp.PrimitiveSet('MAIN_SR', 1)
pset_sr.addPrimitive(operator.add, 2)
pset_sr.addPrimitive(operator.sub, 2)
pset_sr.addPrimitive(operator.mul, 2)
pset_sr.addPrimitive(protectedDiv, 2)
pset_sr.addEphemeralConstant('rand101', lambda: random.uniform(-1, 1))
pset_sr.renameArguments(ARG0='x')
pset_par = gp.PrimitiveSet('MAIN_PAR', 6)
pset_par.addPrimitive(operator.xor, 2)
pset_par.addPrimitive(operator.and_, 2)
pset_par.addPrimitive(operator.or_, 2)
pset_par.addPrimitive(operator.not_, 1)
pset_par.addEphemeralConstant('rand01', lambda: random.choice([0, 1]))
for i in range(6):
    pset_par.renameArguments(**{f'ARG{i}': f'b{i}'})

def sfAdd(i, j):
    return i + j

def sfNeg(i):
    return -i

def ifThen(cond, out1, out2):
    return out1 if cond else out2

pset_sf = gp.PrimitiveSet('MAIN_SF', 1)
pset_sf.addPrimitive(sfAdd, 2)
pset_sf.addPrimitive(sfNeg, 1)
pset_sf.addPrimitive(ifThen, 3)
pset_sf.addEphemeralConstant('randConst', lambda: random.randint(0, 100))
pset_sf.renameArguments(ARG0='step')

C:\Users\JV\AppData\Local\Programs\Python\Python38\lib\site-packages\deap\gp.py:257: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "
C:\Users\JV\AppData\Local\Programs\Python\Python38\lib\site-packages\deap\gp.py:257: RuntimeWarning: Ephemeral rand01 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "
C:\Users\JV\AppData\Local\Programs\Python\Python38\lib\site-packages\deap\gp.py:257: RuntimeWarning: Ephemeral randConst function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


In [5]:
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('IndividualKOZA', gp.PrimitiveTree, fitness=creator.FitnessMin)
creator.create('IndividualPAR', gp.PrimitiveTree, fitness=creator.FitnessMin)
creator.create('IndividualSF', gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox_koza = base.Toolbox()
toolbox_koza.register('expr', gp.genHalfAndHalf, pset=pset_sr, min_=1, max_=3)
toolbox_koza.register('individual', tools.initIterate, creator.IndividualKOZA, toolbox_koza.expr)
toolbox_koza.register('population', tools.initRepeat, list, toolbox_koza.individual)
toolbox_koza.register('compile', gp.compile, pset=pset_sr)

x_vals = np.linspace(-1, 1, 20)

def koza(ind):
    func = toolbox_koza.compile(expr=ind)
    err = 0.0
    for x in x_vals:
        try:
            val = func(x)
            err += (val - (x ** 4 + x ** 3 + x ** 2 + x)) ** 2
        except ZeroDivisionError:
            return float('inf'),
    return err,

toolbox_koza.register('evaluate', koza)
toolbox_koza.register('select', tools.selTournament, tournsize=3)
toolbox_koza.register('mate', gp.cxOnePoint)
toolbox_koza.register('expr_mut', gp.genFull, min_=0, max_=2)
toolbox_koza.register('mutate', gp.mutUniform, expr=toolbox_koza.expr_mut, pset=pset_sr)
toolbox_koza.decorate('mate', gp.staticLimit(key=operator.attrgetter('height'), max_value=MAX_TREE_HEIGHT))
toolbox_koza.decorate('mutate', gp.staticLimit(key=operator.attrgetter('height'), max_value=MAX_TREE_HEIGHT))

In [6]:
toolbox_par = base.Toolbox()
toolbox_par.register('expr', gp.genFull, pset=pset_par, min_=1, max_=3)
toolbox_par.register('individual', tools.initIterate, creator.IndividualPAR, toolbox_par.expr)
toolbox_par.register('population', tools.initRepeat, list, toolbox_par.individual)
toolbox_par.register('compile', gp.compile, pset=pset_par)


def parity(ind):
    func = toolbox_par.compile(expr=ind)
    errs = 0
    for i in range(64):
        bits = [(i >> j) & 1 for j in range(6)]
        if func(*bits) != (sum(bits) % 2):
            errs += 1
    return errs,


toolbox_par.register('evaluate', parity)
toolbox_par.register('select', tools.selTournament, tournsize=3)
toolbox_par.register('mate', gp.cxOnePoint)
toolbox_par.register('expr_mut', gp.genFull, min_=0, max_=2)
toolbox_par.register('mutate', gp.mutUniform, expr=toolbox_par.expr_mut, pset=pset_par)
toolbox_par.decorate('mate', gp.staticLimit(key=operator.attrgetter('height'), max_value=MAX_TREE_HEIGHT))
toolbox_par.decorate('mutate', gp.staticLimit(key=operator.attrgetter('height'), max_value=MAX_TREE_HEIGHT))


def loadSantafeMap():
    rows, sx, sy = [], None, None
    with open('santafe_trail.txt') as f:
        for y, line in enumerate(f):
            row = []
            for x, c in enumerate(line.rstrip("\n")):
                if c == '#':
                    row.append(1)
                elif c in ('.', 'S'):
                    row.append(0)
                    if c == 'S': sx, sy = x, y
            if row: rows.append(row)
    H, W = len(rows), len(rows[0])
    world = [cell for row in rows for cell in row]
    return world, W, H, sx, sy


world, W, H, START_X, START_Y = loadSantafeMap()

toolbox_sf = base.Toolbox()
toolbox_sf.register('expr', gp.genHalfAndHalf, pset=pset_sf, min_=2, max_=6)
toolbox_sf.register('individual', tools.initIterate, creator.IndividualSF, toolbox_sf.expr)
toolbox_sf.register('population', tools.initRepeat, list, toolbox_sf.individual)
toolbox_sf.register('compile', gp.compile, pset=pset_sf)


def santafe(ind):
    func = toolbox_sf.compile(expr=ind)
    x, y, d = START_X, START_Y, 0
    food = 0
    visited = world.copy()
    mask = lambda xx, yy: (yy % H) * W + (xx % W)
    for step in range(400):
        idx = mask(x, y)
        if visited[idx] == 1:
            food += 1
            visited[idx] = 0
        action = func(step) % 3
        if action == 0:
            dx, dy = [(0, 1), (1, 0), (0, -1), (-1, 0)][d]
            x += dx;
            y += dy
        elif action == 1:
            d = (d + 1) % 4
        else:
            d = (d - 1) % 4
    penalty = SF_PEN_LAM * len(ind)
    return -food + penalty,


toolbox_sf.register('evaluate', santafe)
toolbox_sf.register('select', tools.selTournament, tournsize=3)
toolbox_sf.register('mate', gp.cxOnePoint)
toolbox_sf.register('expr_mut', gp.genFull, min_=0, max_=2)
toolbox_sf.register('mutate', gp.mutUniform, expr=toolbox_sf.expr_mut, pset=pset_sf)
toolbox_sf.decorate('mate', gp.staticLimit(key=operator.attrgetter('height'), max_value=MAX_TREE_HEIGHT))
toolbox_sf.decorate('mutate', gp.staticLimit(key=operator.attrgetter('height'), max_value=MAX_TREE_HEIGHT))

In [7]:
def runGP(toolbox, name):
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.HallOfFame(1)
    evals = 0
    best_hist = []
    gen = 0
    while evals < N_EVALS:
        offspring = algorithms.varAnd(pop, toolbox, cxpb=0.9, mutpb=0.1)
        fits = list(map(toolbox.evaluate, offspring))
        for ind, fit in zip(offspring, fits):
            ind.fitness.values = fit
        evals += len(offspring)
        pop = toolbox.select(offspring, k=POP_SIZE)
        hof.update(pop)
        best = hof[0].fitness.values[0]
        best_hist.append(best)
        if gen % 10 == 0:
            print(f'{name} Evals {evals}: Best = {best:.6f}, Len = {len(hof[0])}')
        gen += 1
    return hof[0], best_hist

In [8]:
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(
        csvfile,
        fieldnames=['run_id', 'benchmark', 'replicate', 'seed', 'best', 'len_final', 'cpu_s']
    )
    writer.writeheader()
    benches = {
        'Koza': toolbox_koza,
        'Parity': toolbox_par,
        'SantaFe': toolbox_sf
    }

    for name, tb in benches.items():
        for rep in range(N_REPLICATES):
            seed = random.randint(0, 2 ** 31 - 1)
            random.seed(seed);
            np.random.seed(seed)
            start = time.perf_counter()
            best_ind, hist = runGP(tb, name)
            cpu = time.perf_counter() - start
            writer.writerow({
                'run_id': uuid.uuid4(),
                'benchmark': name,
                'replicate': rep,
                'seed': seed,
                'best': round(best_ind.fitness.values[0], 6),
                'len_final': len(best_ind),
                'cpu_s': round(cpu, 4)
            })

print(f'Results written to {csv_path}')

SantaFe Evals 100: Best = -2.200000, Len = 8
SantaFe Evals 1100: Best = -2.700000, Len = 3
SantaFe Evals 2100: Best = -2.700000, Len = 3
SantaFe Evals 3100: Best = -2.700000, Len = 3
SantaFe Evals 4100: Best = -2.700000, Len = 3
SantaFe Evals 5100: Best = -2.700000, Len = 3
SantaFe Evals 6100: Best = -2.700000, Len = 3
SantaFe Evals 7100: Best = -2.700000, Len = 3
SantaFe Evals 8100: Best = -2.700000, Len = 3
SantaFe Evals 9100: Best = -2.700000, Len = 3
SantaFe Evals 10100: Best = -2.700000, Len = 3
SantaFe Evals 11100: Best = -2.700000, Len = 3
SantaFe Evals 12100: Best = -2.700000, Len = 3
SantaFe Evals 13100: Best = -2.700000, Len = 3
SantaFe Evals 14100: Best = -2.700000, Len = 3
SantaFe Evals 15100: Best = -2.700000, Len = 3
SantaFe Evals 16100: Best = -2.700000, Len = 3
SantaFe Evals 17100: Best = -2.700000, Len = 3
SantaFe Evals 18100: Best = -2.700000, Len = 3
SantaFe Evals 19100: Best = -2.700000, Len = 3
SantaFe Evals 20100: Best = -2.700000, Len = 3
SantaFe Evals 21100: Bes